A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior.

Recurrent networks are distinguished from feedforward networks by that feedback loop, ingesting their own outputs moment after moment as input. It is often said that recurrent networks have memory. Adding memory to neural networks has a purpose: There is information in the sequence itself, and recurrent nets use it to perform tasks that feedforward networks can’t.

Here's a great article: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [3]:
#Import MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
#Setting up tensforflow parameters
learning_rate =  0.001
training_iterations = 100
batch_size = 128

#Network Paramaters
n_hidden = 100 #number of nodes in hidden layer
n_steps = 30 #number of times to iterate over self
n_inputs = 28 #MNIST data 28pixels for each image
n_classes = 10 #Output can be 1 of the 10 numbers


In [18]:
#TF graph inputs

#mnist images are of shape 28*28 = 784
#We do not know the number of images/ they will be different for train and test thus None, 784
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

#y, i.e. the result can be any number from 0 to 9 in onehot encoded format
y = tf.placeholder(tf.float32, [None, n_classes]) 

In [19]:
#Defining weights and bias
W = tf.Variable(tf.random_normal([n_hidden, n_classes]))
b = tf.Variable(tf.random_normal([n_classes]))